# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.experiment import Experiment
import os

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperparameter-tuning-yum'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-223596
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-223596


In [4]:
# validate dataset is in store
key = "Churn"
description_text = "Churn"


if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print('The Dataset was found')
# else:
#     # Create AML Dataset and register it into Workspace
#     data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00592/Churn_Dateset.csv"
#     dataset = Dataset.Tabular.from_delimited_files(data_url)
#     #Register Dataset in Workspace
#     dataset = dataset.register(workspace = ws,name = key,description = description_text)

df = dataset.to_pandas_dataframe()
print(df.head())

   Anonymous Customer ID  Call  Failure  Complains  Subscription  Length  \
0                    1.0            8.0        0.0                  38.0   
1                    2.0            0.0        0.0                  39.0   
2                    3.0           10.0        0.0                  37.0   
3                    4.0           10.0        0.0                  38.0   
4                    5.0            3.0        0.0                  38.0   

   Charge  Amount  Seconds of Use  Frequency of use  Frequency of SMS  \
0             0.0          4370.0              71.0               5.0   
1             0.0           318.0               5.0               7.0   
2             0.0          2453.0              60.0             359.0   
3             0.0          4198.0              66.0               1.0   
4             0.0          2393.0              58.0               2.0   

   Distinct Called Numbers  Age Group  Tariff Plan  Status  Churn  \
0                     17.0        3

## Create or Attach an AmlCompute cluster

In [5]:
# Choose a name for the cluster
cpu_cluster_name = 'cluster-yum'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1, delay_evaluation=5 )

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": choice(0.1, 1, 5),
    "max_iter": choice(25, 50, 100, 200, 500)
}
)
#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=compute_target, environment=sklearn_env)
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name="Accuracy", primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4)

In [12]:
#TODO: Submit your experiment
from azureml.core.experiment import Experiment
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

RunId: HD_5cb6455b-9637-428d-9233-2f63edf95972
Web View: https://ml.azure.com/runs/HD_5cb6455b-9637-428d-9233-2f63edf95972?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-223596/workspaces/quick-starts-ws-223596&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-26T21:32:04.171086][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-26T21:32:05.5193130Z][SCHEDULER][INFO]Scheduling job, id='HD_5cb6455b-9637-428d-9233-2f63edf95972_0' 
[2023-01-26T21:32:05.6560396Z][SCHEDULER][INFO]Scheduling job, id='HD_5cb6455b-9637-428d-9233-2f63edf95972_1' 
[2023-01-26T21:32:05.8017035Z][SCHEDULER][INFO]Scheduling job, id='HD_5cb6455b-9637-428d-9233-2f63edf95972_2' 
[2023-01-26T21:32:05.8776746Z][SCHEDULER][INFO]Scheduling job, id='HD_5cb6455b-9637-428d-9233-2f63edf95972_3' 
[2023-01-26T21:32:05.9099864Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_5cb6455b-9637-428d-9233-2f63edf95972_1

{'runId': 'HD_5cb6455b-9637-428d-9233-2f63edf95972',
 'target': 'cluster-yum',
 'status': 'Completed',
 'startTimeUtc': '2023-01-26T21:32:03.305279Z',
 'endTimeUtc': '2023-01-26T21:34:11.083601Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '25444b2f-44ba-4725-acc2-6a4ffb24451d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '15',
  'score': '0.8670886075949367',
  'best_child_run_id': 'HD_5cb6455b-9637-428d-9233-2f63edf95972_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_5cb6455b-9637-428d-9233-2f63edf95972_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientT

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
import joblib
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
best_run.get_details()

{'runId': 'HD_5cb6455b-9637-428d-9233-2f63edf95972_2',
 'target': 'cluster-yum',
 'status': 'Completed',
 'startTimeUtc': '2023-01-26T21:32:19.664702Z',
 'endTimeUtc': '2023-01-26T21:32:42.500863Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '25444b2f-44ba-4725-acc2-6a4ffb24451d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.1', '--max_iter', '500'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cluster-yum',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  

In [15]:
best_run_metrics = best_run.get_metrics()
print('Accuracy:', best_run_metrics["Accuracy"])

Accuracy: 0.8670886075949367


In [16]:
#TODO: Save the best model
best_run.download_file("model.pkl")

In [17]:
model = best_run.register_model(model_name="model.pkl")
print(model.name, model.id, model.version, sep='\t')

model.pkl	model.pkl:1	1


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

